In [2]:
from qdrant_client import AsyncQdrantClient, models
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from qdrant_client import AsyncQdrantClient
import torch.nn.functional as F
from tools import is_text
import torch
from transformers import AutoTokenizer, AutoModel
import asyncio
import os
from itertools import count
from llama_index.core import VectorStoreIndex
import json
from tqdm import tqdm
from smart_chunker.chunker import SmartChunker
import pickle

/home/cossmo/miniconda3/envs/llama_index_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
REST_API_PORT=6333
EMBEDDINGS_DIM=768
YANDEX_CLOUD_FOLDER = ""
YANDEX_CLOUD_API_KEY = ""
model_name = "gemma-3-27b-it"

In [4]:
docs_config_dir = "doc_base/RuBQ_2.0/"
test_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_test.json')
dev_queries_path = os.path.join(docs_config_dir, 'RuBQ_2.0_dev.json')
paragraphs_path = os.path.join(docs_config_dir, 'RuBQ_2.0_paragraphs.json')

In [5]:
with open(test_queries_path, 'r', encoding='utf-8') as f:
    test_queries = json.load(f)

with open(dev_queries_path, 'r', encoding='utf-8') as f:
    dev_queries = json.load(f)

with open(paragraphs_path, 'r', encoding='utf-8') as f:
    paragraphs = json.load(f)

In [6]:
def create_berta_embeddings(berta_model: AutoModel, 
                            berta_tokenizer: AutoTokenizer, 
                            inputs: list[str], 
                            batch_size=32, 
                            device: str='cuda',
                            prefix: str="search_document: "):
    def pool(hidden_state, mask, pooling_method="mean"):
        if pooling_method == "mean":
            s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
            d = mask.sum(axis=1, keepdim=True).float()
            return s / d
        elif pooling_method == "cls":
            return hidden_state[:, 0]

    # add task prefix if exists:
    if prefix:
        inputs = [prefix + input_str for input_str in inputs]

    batch_count = (len(inputs) + batch_size - 1) // batch_size
    result_embeddings = []
    pbar = tqdm(total=batch_count, desc='creating embeddings...')

    with torch.no_grad():
        for i in range(batch_count):
            batch = inputs[i*batch_size: (i + 1) * batch_size]
            tokenized_inputs = berta_tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors="pt").to(device)
            berta_model.to(device)
            outputs = berta_model(**tokenized_inputs)
            embeddings = pool(
                outputs.last_hidden_state, 
                tokenized_inputs["attention_mask"],
                pooling_method="mean"
            )

            embeddings = F.normalize(embeddings, p=2, dim=1).to('cpu')
            result_embeddings.append(embeddings)
            pbar.update(1)
    result_embeddings = torch.cat(result_embeddings, dim=0)
    print(f'embeddings count={len(result_embeddings)}', flush=True)
    
    return result_embeddings


def load_model(model_name:str="sergeyzh/BERTA"):
    print('loading model and tokenizer...', flush=True)
    try:
        berta_tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=True)
        berta_model = AutoModel.from_pretrained(model_name, local_files_only=True)
    except Exception as e:
        print('no local files found, load from server...', flush=True)
        berta_tokenizer = AutoTokenizer.from_pretrained(model_name)
        berta_model = AutoModel.from_pretrained(model_name)

    return berta_model, berta_tokenizer


# split large paragraphs into chunks with smart-chunker
def chunk_large_paragraphs(paragraphs: dict, tokenizer: AutoTokenizer, max_tokens=512):
    chunker = SmartChunker(
        language='ru',
        reranker_name='BAAI/bge-reranker-v2-m3',
        newline_as_separator=False,
        device='cuda:0'
    )
    # split large paragraph into smaller
    def split_paragraph(paragraph):
        nonlocal chunker
        texts = chunker.split_into_chunks(paragraph['text'])

        return [{'uid':paragraph['uid'], 'text':text, 'ru_wiki_pageid': paragraph['ru_wiki_pageid']} for text in texts]

    result = []
    texts = [paragraph['text'] for paragraph in paragraphs]
    tokenized = tokenizer(texts, truncation=False)

    token_counts = [len(tokens) for tokens in tokenized['input_ids']]
    pbar=tqdm(total=len(paragraphs), desc='Chunking large paragraphs...')

    for i in range(len(paragraphs)):
        token_count = token_counts[i]
        paragraph = paragraphs[i]

        if token_count > max_tokens:
            new_paragraphs = split_paragraph(paragraph)
            for n_par in new_paragraphs:
                result.append(n_par)
        else:
            result.append(paragraph)

        pbar.update(1)

    return result


async def create_paragraphs_database(qdrant_client: AsyncQdrantClient,
                                     paragraphs:dict,
                                     collection:str='paragraphs',
                                     batch_size:int=32,
                                     chunk_large:bool=False,
                                     cache_embeddings:bool=True,
                                     use_cached_embeddings:bool=False):
    if not qdrant_client:
        return
    # check collection already created
    if await qdrant_client.collection_exists(collection):
        return
    await qdrant_client.create_collection(collection_name=collection,
                                          vectors_config=models.VectorParams(size=EMBEDDINGS_DIM, distance=models.Distance.COSINE)
                                          )
    berta_model, berta_tokenizer = load_model()
    # chunk large paragraphs if required
    if chunk_large:
        paragraphs = chunk_large_paragraphs(paragraphs, berta_tokenizer)

    texts = [paragraph['text'] for paragraph in paragraphs]
    batch_size = 32

    if use_cached_embeddings:
        # load checkpoint embeddings:
        if not os.path.isfile('embeddings.pkl'):
            raise Exception('Trying to load cached embeddings - no cached embeddings file found')
        print("load cached embeddings...")
        with open('embeddings.pkl', 'rb') as f:
            embeddings = pickle.load(f)
    else: 
        # create from scratch:
        embeddings = create_berta_embeddings(berta_model, berta_tokenizer, texts, batch_size=batch_size)

    # save embeddings in case of cache
    if cache_embeddings and not use_cached_embeddings:
        with open('embeddings.pkl', 'wb') as f:
            pickle.dump(embeddings, f)

    id_counter = count(start=0)    # global chunk id in qdrant database

    # embeddings and corresponding paragraphs iterator:
    def batch_iterator(paragraphs, embeddings, batch_size=32):
        for i in range(0, len(paragraphs), batch_size):
            yield (paragraphs[i: i + batch_size], embeddings[i:i + batch_size])

    pabar=tqdm(total=(len(paragraphs) + batch_size - 1) // batch_size, desc='loading messages to Qdrant storage...')
    batch_idx=0

    # insert chunks to qdrant base:
    for batch in batch_iterator(paragraphs, embeddings, batch_size):
        paragraphs, embeddings = batch
        operation_info = await qdrant_client.upsert(collection_name=collection,
                                                    points=[models.PointStruct(id=next(id_counter),
                                                                                vector=embeddings[i],
                                                                                payload={"paragraph_id": paragraphs[i]['uid'], 
                                                                                         "text": paragraphs[i]['text']
                                                                                         }
                                                                                )
                                                            for i in range(len(embeddings))
                                                            ]
                                                    )
        if operation_info.status == models.UpdateStatus.ACKNOWLEDGED:
            print(f'WARNING: acknowledged request on batch - {batch_idx}', flush=True)
        batch_idx += 1
        pabar.update(1)

    berta_model.to('cpu')
    del berta_model


async def create_chunked_database(qdrant_client: AsyncQdrantClient,
                                  documents_dir: str='data/', 
                                  chunk_splitter:str='\n'*4, 
                                  collection:str='nsu_base'
                                  ):
    if not qdrant_client:
        return
    # check collection already created
    if await qdrant_client.collection_exists(collection):
        return
    await qdrant_client.create_collection(collection_name=collection,
                                          vectors_config=models.VectorParams(size=EMBEDDINGS_DIM, distance=models.Distance.COSINE)
                                          )
    berta_model, berta_tokenizer = load_model()
    id_counter = count(start=0)    # global chunk id in qdrant database

    for file in tqdm(filter(lambda f: is_text(f), os.listdir(documents_dir)), desc="inserting chunks..."):
        f_path = os.path.join(documents_dir, file)
        with open(f_path, encoding='utf-8') as f:
            data = f.read()

        # create embeddings of document's chunks:
        chunks = data.split(chunk_splitter)
        embeddings = create_berta_embeddings(berta_model, berta_tokenizer, chunks)
        local_id_counter = count(start=0)  # chunk id inside document

        # insert embeddings to qdrant base:
        operation_info = await qdrant_client.upsert(collection_name=collection,
                                                    points=[models.PointStruct(id=next(id_counter),
                                                                               vector=embeddings[i],
                                                                               payload={"doc_name": file, "chunk_id": next(local_id_counter), "text": chunks[i]}
                                                                               )
                                                            for i in range(len(embeddings))
                                                            ]
                                                    )
        if operation_info.status == models.UpdateStatus.ACKNOWLEDGED:
            print(f'WARNING: acknowledged request for doc - {file}', flush=True)
    berta_model.to('cpu')
    del berta_model


### Смотрим распределение числа токенов в параграфах

In [7]:
import plotly.graph_objects as go
from typing import List

berta_tokenizer = AutoTokenizer.from_pretrained("sergeyzh/BERTA", local_files_only=True)

def plot_tokens_dist(paragraphs, tokenizer):
    texts = [paragraph['text'] for paragraph in paragraphs]
# Example: suppose you have token counts for each text
    def get_tokens_distribution(tokenizer: AutoTokenizer, texts: List[str]):
        tokenized = tokenizer(texts, truncation=False)

        return [len(tokens) for tokens in tokenized['input_ids']]

    tokens_count = get_tokens_distribution(tokenizer, texts)

    fig = go.Figure(
        data=[go.Histogram(x=tokens_count, nbinsx=50, marker=dict(line=dict(width=1, color="black")))]
    )

    # threshold:
    fig.add_shape(
        type="line",
        x0=512,
        x1=512,
        y0=0,
        y1=max(tokens_count),  # or you can use fig.data[0].y.max() after rendering
        line=dict(color="red", width=2, dash="dash")
    )
    fig.update_layout(
        title="Token Count Distribution",
        xaxis_title="Number of Tokens",
        yaxis_title="Frequency",
        bargap=0.1,
    )    

    fig.show()

### Распределение числа токенов для исходных параграфов

In [8]:
plot_tokens_dist(paragraphs, berta_tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


### После чанкинга

In [9]:
load_chunks=True
if load_chunks:
    with open('doc_base/chunked_paragraphs.json', 'r', encoding='utf-8') as f:
        chunked_large_paragraphs = json.loads(f.read())
else:
    chunked_large_paragraphs = chunk_large_paragraphs(paragraphs, berta_tokenizer)

In [10]:
plot_tokens_dist(chunked_large_paragraphs, berta_tokenizer)

In [9]:
with open('doc_base/chunked_paragraphs.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(chunked_large_paragraphs))

In [11]:
chunked_large_paragraphs[0]

{'uid': 0,
 'ru_wiki_pageid': 58311,
 'text': 'ЦСКА — советский и российский профессиональный хоккейный клуб из Москвы, выступающий в Континентальной хоккейной лиге. Основан в 1946 году под названием ЦДКА (Центральный дом Красной Армии). В 1951 году переименован в ЦДСА (Центральный дом Советской Армии), а в 1954 в ЦСК МО (Центральный спортивный клуб Министерства обороны), под которым выступал до 1959 года, и с тех пор носит название ЦСКА (Центральный Спортивный Клуб Армии).'}

### Посмотрим как отработало разбиение на чанки

In [12]:
def get_tokens_count(paragraphs: dict, tokenizer: AutoTokenizer):
    texts = [paragraph['text'] for paragraph in paragraphs]
# Example: suppose you have token counts for each text
    def get_tokens_distribution(tokenizer: AutoTokenizer, texts: List[str]):
        tokenized = tokenizer(texts, truncation=False)

        return [len(tokens) for tokens in tokenized['input_ids']]

    tokens_count = get_tokens_distribution(tokenizer, texts)

    return tokens_count

In [13]:
tokens_count = get_tokens_count(paragraphs, berta_tokenizer)
large_paragraphs = [paragraphs[i] for i in range(len(paragraphs)) if tokens_count[i] > 512]

st_pos=10
end_pos=10

for large in large_paragraphs[st_pos: end_pos + 1]:
    print('Original:\n')
    print(large['text'])
    print('#' * 40)
    print('Chunked:\n')

    splitted = [chunked for chunked in chunked_large_paragraphs if chunked['uid'] == large['uid']]

    for chunk in splitted:
        print(chunk['text'])
        print('\n' + '-'*40)

Original:

Валуйский приводит сведения из статьи Ш. Чухо в сочинской газете «Красное знамя» от 27 марта 1960 г. о том, что одна из форм названия р. Сочи — Сшычье (в исторических источниках неизвестная). По свидетельству Ш.Чухо, в старошапсугском диалекте сши означает «брат», а чье — «без», то есть «река без притока». Однако такой перевод относится к образцу «народных этимологий». Если переводить с шапсугского диалекта, то надо исходить и из шапсугских же вариантов названия: Сша-ше, Шаше, Сфеши. К тому же река Сочи уже в устье имеет правый приток с широкой долиной, ручей Хлудовского (Шлабистага), не говоря о других (Ац, Агва, Ушха, Ажек и пр.). М. В. Валуйский сообщает также, что у черноморских шапсугов было распространено неправильное мнение (основанное на сходстве звучания) о том, что название Сочи связано с адыгейским наименованием конных скачек, так называемых Шъаче (на самом деле по-адыг. скачки — шыгъачьэ), которые прежде осуществлялись на обширной равнине левого приустья реки Соч

### Создаём Qdrant-client и базу

In [21]:
base_name = 'paragraphs'
query_prefix = "search_query: "
qdrant_client = AsyncQdrantClient(url=f"http://localhost:{REST_API_PORT}")

In [22]:
qdrant_client

In [ ]:
await create_paragraphs_database(qdrant_client, paragraphs, batch_size=64, use_cached_embeddings=True)
vector_store = QdrantVectorStore(aclient=qdrant_client, collection_name=base_name)

### Задаём свою эмбеддер-модель

In [24]:
# add prefix for query embeddings calculation:
embed_model = HuggingFaceEmbedding(model_name="sergeyzh/BERTA", device='cuda', query_instruction=query_prefix)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, 
                                           embed_model=embed_model
                                          )

Default prompt name is set to 'Classification'. This prompt will be applied to all `encode()` calls, except if `encode()` is called with `prompt` or `prompt_name` parameters.


### Доступ к llm по openAI API

In [25]:
from llama_index.llms.openai_like import OpenAILike


llm = OpenAILike(
    model=f"gpt://{YANDEX_CLOUD_FOLDER}/{model_name}",
    api_base="https://llm.api.cloud.yandex.net/v1",
    api_key=YANDEX_CLOUD_API_KEY,
    is_chat_model=True  # set as needed for your model
)
query_engine = index.as_query_engine(llm=llm)

### Кастомизируем промпт

In [ ]:
# TODO: поменять дефолтный промпт для LLM

### Тестируем наш RAG

In [19]:
queries_count=10
queries = [query['question_text'] for query in dev_queries[:queries_count]]

In [28]:
for query in queries:
    print(f'Question: {query}')
    resp = await query_engine.aquery(query)
    print(f'Response: {str(resp)}')
    print('-'*40)

Question: Какой стране принадлежит знаменитый остров Пасхи?
Response: Остров Пасхи принадлежит Чили.
----------------------------------------
Question: С какой музыкальной группой неразрывно связано имя Мика Джаггера?
Response: Мик Джаггер является вокалистом рок-группы The Rolling Stones.
----------------------------------------
Question: Где находится Летний сад?
Response: Летний сад находится в историческом центре Кронштадта, на Петровской улице.
----------------------------------------
Question: Какой город является столицей Туркмении?
Response: Ашхабад является столицей Туркмении.
----------------------------------------
Question: В каком городе издавалась с 1857 г. А. Герценом и Н. Огаревым первая российская революционная газета "Колокол"?
Response: Изначально газета издавалась в Лондоне, но впоследствии была перенесена в Женеву.
----------------------------------------
Question: В какой стране находится второй из самых высоких действующих вулканов с забавным названием Попокатепе

In [ ]:
# prompts = query_engine.get_prompts()
# prompts

{'response_synthesizer:text_qa_template': SelectorPromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings={}, function_mappings={}, default_template=PromptTemplate(metadata={'prompt_type': <PromptType.QUESTION_ANSWER: 'text_qa'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='Context information is below.\n---------------------\n{context_str}\n---------------------\nGiven the context information and not prior knowledge, answer the query.\nQuery: {query_str}\nAnswer: '), conditionals=[(<function is_chat_model at 0x776340b86d40>, ChatPromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['context_str', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, message_templates=[ChatMessage(role=<MessageRole.

### Логирование

In [29]:
def log_resp(resp):
    selected_chunks = resp.source_nodes
    print(f"chunks count={len(selected_chunks)}")
    
    for chunk in selected_chunks:
        metadata = chunk.metadata
        doc_name = metadata.get("doc_name")
        chunk_id = metadata.get("chunk_id")
        chunk_text = chunk.text
        
        # Log or print the chunk information
        print(f"Document Name: {doc_name}, Chunk ID: {chunk_id}, Chunk Text: \n{chunk_text}") 
        print('-'*50)

In [30]:
log_resp(resp)

chunks count=2
Document Name: None, Chunk ID: None, Chunk Text: 
Сэ́мюэл Фи́нли Бриз Мо́рзе (англ. Samuel Finley Breese Morse [mɔːrs]; 27 апреля 1791, Чарльзтаун в штате Массачусетс — 2 апреля 1872, Нью-Йорк) — американский изобретатель и художник. Наиболее известные изобретения — электромагнитный пишущий телеграф («аппарат Морзе», 1836) и код (азбука) Морзе.
--------------------------------------------------
Document Name: None, Chunk ID: None, Chunk Text: 
Газеты, железные дороги и банки быстро нашли применение его телеграфу. Телеграфные линии моментально оплели весь мир, состояние и слава Морзе умножились. В 1858 году от десяти европейских государств Морзе получил за своё изобретение 400 000 франков. Морзе купил имение в Покипси, близ Нью-Йорка, и провёл там остаток жизни с большим семейством среди детей и внуков. В старости Морзе стал филантропом. Он опекал школы, университеты, церкви, библейские общества, миссионеров и бедных художников.
-------------------------------------------